Если хотите поэкспериментировать дальше, можете попробовать:

n_estimators=2000, learning_rate=0.01, num_leaves=70, min_child_samples=40
n_estimators=4000, learning_rate=0.007, num_leaves=90, min_child_samples=60, reg_alpha=0.2, reg_lambda=0.2


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from lightgbm.basic import LightGBMError
import re

In [2]:
# --- Функция для очистки имен признаков ---
def clean_feature_names(feature_names):
    """
    Очищает имена признаков, удаляя или заменяя специальные символы,
    которые могут вызвать проблемы в LightGBM.
    Оставляет только буквы, цифры и подчеркивания.
    """
    cleaned_names = []
    for name in feature_names:
        # Заменяем все, что НЕ является буквой (a-z, A-Z), цифрой (0-9), подчеркиванием (_), или пробелом (\s).
        # Сначала заменяем спецсимволы на '_'.
        cleaned_name = re.sub(r'[^\w\s]', '_', name)
        # Затем заменяем один или несколько пробелов на один '_'.
        cleaned_name = re.sub(r'\s+', '_', cleaned_name)
        # Убираем множественные подчеркивания.
        cleaned_name = re.sub(r'_{2,}', '_', cleaned_name)
        # Убираем подчеркивание в начале/конце, если оно есть.
        cleaned_name = cleaned_name.strip('_')
        # Если после очистки имя стало пустым, генерируем безопасное имя.
        if not cleaned_name:
            cleaned_name = f"unnamed_feature_{hash(name) % 1000}"
        cleaned_names.append(cleaned_name)
    return np.array(cleaned_names)


In [3]:
# --- Загрузка данных ---
train_df = pd.read_csv('Video_Games.csv')
test_df = pd.read_csv('Video_Games_Test.csv')

test_ids = test_df['Id']
train_jp_sales = train_df['JP_Sales']
train_df = train_df.drop('JP_Sales', axis=1)

train_df['Is_Train'] = 1
test_df['Is_Train'] = 0
if 'Id' in test_df.columns:
    test_df = test_df.drop('Id', axis=1)

combined_df = pd.concat([train_df, test_df], ignore_index=True)


In [4]:
# --- Предобработка данных и Инженерия признаков ---

# User_Score: 'tbd' -> NaN -> numeric
combined_df['User_Score'] = combined_df['User_Score'].replace('tbd', np.nan)
combined_df['User_Score'] = pd.to_numeric(combined_df['User_Score'])

# Total_Sales
combined_df['Total_Sales'] = combined_df['NA_Sales'] + combined_df['EU_Sales'] + combined_df['Other_Sales']

# Age_of_Game
current_year = 2024
combined_df['Age_of_Game'] = current_year - combined_df['Year_of_Release']
combined_df['Age_of_Game'] = combined_df['Age_of_Game'].apply(lambda x: max(x, 0) if pd.notna(x) else x)
combined_df = combined_df.drop('Year_of_Release', axis=1)

# Is_Adult_Rating
adult_ratings = ['M', 'AO']
combined_df['Is_Adult_Rating'] = combined_df['Rating'].apply(lambda x: 1 if x in adult_ratings else 0)


In [5]:
# --- Новые признаки: Платформы ---
# Получаем список уникальных платформ ИЗ ОБЪЕДИНЕННОГО датафрейма
all_platforms = combined_df['Platform'].unique()

# Создаем бинарные признаки для каждой платформы, сразу очищая имена
platform_binary_features = []
for platform in all_platforms:
    # Очищаем имя платформы: заменяем спецсимволы и пробелы на подчеркивания
    cleaned_platform_name = re.sub(r'[^\w\s]', '_', platform) # Заменяем спецсимволы на '_'
    cleaned_platform_name = re.sub(r'\s+', '_', cleaned_platform_name) # Заменяем пробелы на '_'
    cleaned_platform_name = cleaned_platform_name.strip('_') # Убираем подчеркивания в начале/конце
    cleaned_platform_name = re.sub(r'_{2,}', '_', cleaned_platform_name) # Убираем множественные подчеркивания

    # Если имя стало пустым после очистки, генерируем безопасное имя
    if not cleaned_platform_name:
        cleaned_platform_name = f"unnamed_platform_{hash(platform) % 1000}"

    platform_binary_features.append(f'Is_{cleaned_platform_name}')

# Важно: После создания бинарных признаков для платформ,
# исходный столбец 'Platform' больше не нужен.
# Удаляем его, чтобы он не попал в 'remainder'.
if 'Platform' in combined_df.columns:
    combined_df = combined_df.drop('Platform', axis=1)

# Удаляем столбец 'Name', так как он не является признаком для модели
if 'Name' in combined_df.columns:
    combined_df = combined_df.drop('Name', axis=1)

In [6]:
# --- Исследование категориальных признаков ---
categorical_features_for_analysis = ['Genre', 'Publisher', 'Developer', 'Rating']
print("\n--- Исследование категориальных признаков ---")
for col in categorical_features_for_analysis:
    if col in combined_df.columns:
        print(f"Признак: '{col}'")
        print(f"  Уникальных значений: {combined_df[col].nunique()}")
        print(f"  Количество NaN: {combined_df[col].isnull().sum()}")
        # Проверка на пустые строки, если это строковый тип
        if combined_df[col].dtype == 'object':
            empty_string_count = combined_df[col].apply(lambda x: isinstance(x, str) and x.strip() == '').sum()
            if empty_string_count > 0:
                print(f"  ВНИМАНИЕ: Признак '{col}' содержит {empty_string_count} пустых строк!")
    else:
        print(f"Признак '{col}' отсутствует в combined_df.")



--- Исследование категориальных признаков ---
Признак: 'Genre'
  Уникальных значений: 12
  Количество NaN: 2
Признак: 'Publisher'
  Уникальных значений: 580
  Количество NaN: 54
Признак: 'Developer'
  Уникальных значений: 1696
  Количество NaN: 6623
Признак: 'Rating'
  Уникальных значений: 8
  Количество NaN: 6769


In [7]:
# --- Определение признаков для модели ---
categorical_features = ['Genre', 'Publisher', 'Developer', 'Rating']
categorical_features = [col for col in categorical_features if col in combined_df.columns]

new_binary_features = ['Is_Adult_Rating']
# Используем уже очищенные имена платформ
numerical_features = [
    'NA_Sales', 'EU_Sales', 'Other_Sales', 'Critic_Score', 'Critic_Count',
    'User_Score', 'User_Count', 'Total_Sales', 'Age_of_Game'
] + new_binary_features + platform_binary_features

# Убедимся, что все перечисленные признаки действительно существуют в DataFrame
numerical_features = [col for col in numerical_features if col in combined_df.columns]


In [8]:
# --- Создание пайплайнов для предобработки ---

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Импутация категориальных признаков: заполняем NaN и пустые строки 'missing'
categorical_imputer = SimpleImputer(strategy='constant', fill_value='missing', missing_values=np.nan)

# Обработка пустых строк перед импутацией
for col in categorical_features:
    if col in combined_df.columns and combined_df[col].dtype == 'object':
        empty_string_mask = combined_df[col].apply(lambda x: isinstance(x, str) and x.strip() == '')
        if empty_string_mask.any():
            combined_df.loc[empty_string_mask, col] = 'missing'

# OneHotEncoder: sparse_output=False для плотного выхода, handle_unknown='ignore'
categorical_transformer = Pipeline(steps=[
    ('imputer', categorical_imputer),
])

In [9]:
# --- Создание ColumnTransformer ---
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough', # Оставляем 'Is_Train'
    sparse_threshold=0 # Гарантирует плотный выход
)

In [10]:
# --- Применение предобработки ---
combined_processed = preprocessor.fit_transform(combined_df)

In [11]:
# --- Получение имен признаков после обработки ---

num_feature_names_out = numerical_features

# Генерация имен для категориальных признаков после OHE
ohe_cat_names_generator = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
if categorical_features:
    sample_cat_data_for_fit = combined_df[categorical_features].copy()
    sample_cat_data_for_fit = sample_cat_data_for_fit.fillna('missing')
    for col in categorical_features:
         if col in sample_cat_data_for_fit.columns and sample_cat_data_for_fit[col].dtype == 'object':
            sample_cat_data_for_fit[col] = sample_cat_data_for_fit[col].apply(lambda x: str(x).strip() if isinstance(x, str) else x)
            sample_cat_data_for_fit[col] = sample_cat_data_for_fit[col].replace('', 'missing')

    ohe_cat_names_generator.fit(sample_cat_data_for_fit)
    cat_feature_names_out = ohe_cat_names_generator.get_feature_names_out(categorical_features)
else:
    cat_feature_names_out = np.array([])

# Имена признаков, оставшихся в remainder
processed_cols_in_transformers_set = set(numerical_features + categorical_features)
passthrough_feature_names = [col for col in combined_df.columns if col not in processed_cols_in_transformers_set]

# --- Отладочный вывод для проверки ---
print("\n--- Детальная проверка имен признаков ---")
print(f"Количество num_feature_names_out: {len(num_feature_names_out)}")
print(f"Количество cat_feature_names_out: {len(cat_feature_names_out)}")
print(f"Количество passthrough_feature_names: {len(passthrough_feature_names)}")
print(f"Список passthrough_feature_names: {passthrough_feature_names}")


--- Детальная проверка имен признаков ---
Количество num_feature_names_out: 10
Количество cat_feature_names_out: 2300
Количество passthrough_feature_names: 1
Список passthrough_feature_names: ['Is_Train']


In [12]:
# --- Анализ выходных столбцов ColumnTransformer ---
actual_num_cols = len(num_feature_names_out)
actual_cat_cols = len(cat_feature_names_out)
actual_passthrough_cols = len(passthrough_feature_names)

print("\n--- Анализ выходных столбцов ColumnTransformer ---")
print(f"  num: {actual_num_cols} столбцов")
print(f"  cat: {actual_cat_cols} столбцов")
print(f"  remainder: {actual_passthrough_cols} столбцов")
print(f"Общее количество столбцов, ожидаемое от ColumnTransformer (суммируя компоненты): {actual_num_cols + actual_cat_cols + actual_passthrough_cols}")
print(f"Общее количество столбцов, фактически полученное (combined_processed.shape[1]): {combined_processed.shape[1]}")



--- Анализ выходных столбцов ColumnTransformer ---
  num: 10 столбцов
  cat: 2300 столбцов
  remainder: 1 столбцов
Общее количество столбцов, ожидаемое от ColumnTransformer (суммируя компоненты): 2311
Общее количество столбцов, фактически полученное (combined_processed.shape[1]): 15


In [13]:
# --- Формирование all_feature_names ---
all_feature_names = np.concatenate([
    num_feature_names_out,
    cat_feature_names_out,
    passthrough_feature_names
])

In [14]:
# --- Очистка имен признаков ---
original_feature_names = all_feature_names.copy()  # Сохраняем оригиналы для отладки
all_feature_names = clean_feature_names(all_feature_names)

# Проверка на дубликаты в именах признаков
unique_feature_names, counts = np.unique(all_feature_names, return_counts=True)
duplicates = unique_feature_names[counts > 1]

if len(duplicates) > 0:
    print(f"\n!!! ОБНАРУЖЕНЫ ДУБЛИКАТЫ В ИМЕНАХ ПРИЗНАКОВ: {duplicates} !!!")
    print("Добавляем суффиксы для обеспечения уникальности...")

    # Создаем словарь для отслеживания счетчиков для каждого дублирующегося имени
    name_counts = {}
    final_cleaned_names = []

    for name in all_feature_names:
        if name in duplicates:
            # Если имя дублируется, добавляем суффикс
            count = name_counts.get(name, 0)
            final_cleaned_names.append(f"{name}_{count}")
            name_counts[name] = count + 1
        else:
            # Если имя уникально, добавляем как есть
            final_cleaned_names.append(name)
    
    all_feature_names = np.array(final_cleaned_names)
    print("Имена признаков были уникализированы.")
else:
    print("\nДубликатов в именах признаков не обнаружено.")

# --- Проверка на совпадение длины имен признаков и количества столбцов после ColumnTransformer ---
print(f"\nКоличество признаков (all_feature_names): {len(all_feature_names)}")
print(f"Количество столбцов в обработанных данных: {combined_processed.shape[1]}")

if len(all_feature_names) != combined_processed.shape[1]:
    print("\n!!! РАЗНИЦА В КОЛИЧЕСТВЕ СТОЛБЦОВ ПОСЛЕ ОЧИСТКИ ИМЕН !!!")
    print(f"Ожидаемое количество столбцов: {len(all_feature_names)}")
    print(f"Фактическое количество столбцов: {combined_processed.shape[1]}")
    print("Это критическая ошибка. Количество имен признаков должно точно совпадать.")
else:
    # Создаем DataFrame из обработанных данных
    try:
        processed_df = pd.DataFrame(combined_processed, columns=all_feature_names)
        print("\nDataFrame успешно создан.")
    except Exception as e:
        print(f"Ошибка при создании DataFrame: {e}")


!!! ОБНАРУЖЕНЫ ДУБЛИКАТЫ В ИМЕНАХ ПРИЗНАКОВ: ['Developer_Full_Fat' 'Publisher_Milestone_S_r_l'] !!!
Добавляем суффиксы для обеспечения уникальности...
Имена признаков были уникализированы.

Количество признаков (all_feature_names): 2311
Количество столбцов в обработанных данных: 15

!!! РАЗНИЦА В КОЛИЧЕСТВЕ СТОЛБЦОВ ПОСЛЕ ОЧИСТКИ ИМЕН !!!
Ожидаемое количество столбцов: 2311
Фактическое количество столбцов: 15
Это критическая ошибка. Количество имен признаков должно точно совпадать.


In [16]:
# Проверим структуру данных до применения ColumnTransformer
print(f"Количество признаков в combined_df: {combined_df.shape[1]}")
print(f"Список признаков: {combined_df.columns.tolist()}")

Количество признаков в combined_df: 15
Список признаков: ['Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'Other_Sales', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating', 'Is_Train', 'Total_Sales', 'Age_of_Game', 'Is_Adult_Rating']


In [17]:
# Проверим, что все нужные признаки включены в ColumnTransformer
print(f"Числовые признаки (numerical_features): {numerical_features}")
print(f"Категориальные признаки (categorical_features): {categorical_features}")
print(f"Признаки платформ (platform_binary_features): {platform_binary_features}")

Числовые признаки (numerical_features): ['NA_Sales', 'EU_Sales', 'Other_Sales', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Total_Sales', 'Age_of_Game', 'Is_Adult_Rating']
Категориальные признаки (categorical_features): ['Genre', 'Publisher', 'Developer', 'Rating']
Признаки платформ (platform_binary_features): ['Is_PSP', 'Is_WiiU', 'Is_PS2', 'Is_GBA', 'Is_Wii', 'Is_PC', 'Is_XB', 'Is_PS', 'Is_XOne', 'Is_PSV', 'Is_PS3', 'Is_X360', 'Is_PS4', 'Is_SNES', 'Is_3DS', 'Is_N64', 'Is_2600', 'Is_GC', 'Is_DS', 'Is_DC', 'Is_GB', 'Is_SAT', 'Is_NES', 'Is_GEN', 'Is_SCD', 'Is_NG', 'Is_WS', 'Is_GG', 'Is_TG16', 'Is_3DO', 'Is_PCFX']


In [18]:
# Применяем ColumnTransformer и смотрим на размер данных
combined_processed = preprocessor.fit_transform(combined_df)
print(f"Размер данных после ColumnTransformer: {combined_processed.shape}")

# Убедимся, что количество столбцов в combined_processed совпадает с количеством имен признаков
print(f"Ожидаемое количество признаков: {len(all_feature_names)}")
print(f"Фактическое количество столбцов: {combined_processed.shape[1]}")

Размер данных после ColumnTransformer: (16719, 15)
Ожидаемое количество признаков: 2311
Фактическое количество столбцов: 15


In [ ]:
# --- Проверка на дубликаты в именах признаков ---
# LightGBM не поддерживает дубликаты в именах признаков
unique_feature_names, counts = np.unique(all_feature_names, return_counts=True)
duplicates = unique_feature_names[counts > 1]

if len(duplicates) > 0:
    print(f"\n!!! ОБНАРУЖЕНЫ ДУБЛИКАТЫ В ИМЕНАХ ПРИЗНАКОВ: {duplicates} !!!")
    print("Добавляем суффиксы для обеспечения уникальности...")

    # Создаем словарь для отслеживания счетчиков для каждого дублирующегося имени
    name_counts = {}
    final_cleaned_names = []

    for name in all_feature_names:
        if name in duplicates:
            # Если имя дублируется, добавляем суффикс
            count = name_counts.get(name, 0)
            final_cleaned_names.append(f"{name}_{count}")
            name_counts[name] = count + 1
        else:
            # Если имя уникально, добавляем как есть
            final_cleaned_names.append(name)
    all_feature_names = np.array(final_cleaned_names)
    print("Имена признаков были уникализированы.")
else:
    print("\nДубликатов в именах признаков не обнаружено.")

In [ ]:
# --- Проверка размеров ---
print(f"\nОбщее количество имен признаков (all_feature_names): {len(all_feature_names)}")
print(f"Количество столбцов в combined_processed: {combined_processed.shape[1]}")

if len(all_feature_names) != combined_processed.shape[1]:
    print("\n!!! РАЗНИЦА В КОЛИЧЕСТВЕ СТОЛБЦОВ ПОСЛЕ ОЧИСТКИ ИМЕН !!!")
    print("Это критическая ошибка. Количество имен признаков должно точно совпадать.")
else:
    # Создаем DataFrame из обработанных данных
    try:
        processed_df = pd.DataFrame(combined_processed, columns=all_feature_names)
        print("\nDataFrame успешно создан.")

        if 'Is_Train' in processed_df.columns:
            train_processed_df = processed_df[processed_df['Is_Train'] == 1].drop('Is_Train', axis=1)
            test_processed_df = processed_df[processed_df['Is_Train'] == 0].drop('Is_Train', axis=1)
            print("Данные разделены обратно на train и test.")
        else:
            print("\n!!! ОШИБКА: Столбец 'Is_Train' отсутствует в processed_df !!!")

        train_processed_df['JP_Sales'] = train_jp_sales

        X_train = train_processed_df.drop('JP_Sales', axis=1)
        y_train = train_processed_df['JP_Sales']
        X_test = test_processed_df

        # --- Выбор и обучение модели ---
        #params = {
        #    'n_estimators': 4000,
        #    'learning_rate': 0.008,
        #    'num_leaves': 80,
        #   'max_depth': -1,
        #    'min_child_samples': 40,
        #    'subsample': 0.8,
        #    'colsample_bytree': 0.8,
        #    'reg_alpha': 0.1,
        #    'reg_lambda': 0.1,
        #    'n_jobs': -1,
        #    'metric': 'mae',
        #    'boosting_type': 'gbdt',
        #    'random_state': 42
        #}

        #params = {
        #    'n_estimators': 5000,         # Увеличиваем количество деревьев
        #    'learning_rate': 0.007,       # Слегка уменьшаем learning rate для более точного обучения
        #    'num_leaves': 100,            # Увеличиваем num_leaves для более сложных зависимостей
        #    'max_depth': -1,              # Без ограничения глубины (но num_leaves уже ограничивает сложность)
        #    'min_child_samples': 50,      # Увеличиваем min_child_samples для снижения переобучения
        #    'subsample': 0.8,             # Оставляем прежним
        #    'colsample_bytree': 0.8,      # Оставляем прежним
        #    'reg_alpha': 0.1,             # L1 регуляризация
        #    'reg_lambda': 0.1,            # L2 регуляризация
        #    'n_jobs': -1,                 # Использовать все доступные ядра CPU
        #    'metric': 'mae',              # Метрика для оценки
        #    'boosting_type': 'gbdt',
        #    'random_state': 42
        #} #MAE на полном обучающем наборе (после добавления признаков): 0.0632

        params = {
            'n_estimators': 3000,         # Увеличиваем количество деревьев
            'learning_rate': 0.005,       # Слегка уменьшаем learning rate для более точного обучения
            'num_leaves': 100,            # Увеличиваем num_leaves для более сложных зависимостей
            'max_depth': -1,              # Без ограничения глубины (но num_leaves уже ограничивает сложность)
            'min_child_samples': 50,      # Увеличиваем min_child_samples для снижения переобучения
            'subsample': 0.8,             # Оставляем прежним
            'colsample_bytree': 0.8,      # Оставляем прежним
            'reg_alpha': 0.1,             # L1 регуляризация
            'reg_lambda': 0.1,            # L2 регуляризация
            'n_jobs': -1,                 # Использовать все доступные ядра CPU
            'metric': 'mae',              # Метрика для оценки
            'boosting_type': 'gbdt',
            'random_state': 42
        } #MAE на полном обучающем наборе (после добавления признаков): 0.0633

        lgbm_model = lgb.LGBMRegressor(**params)

        X_train_part, X_val, y_train_part, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        lgbm_model.fit(X_train_part, y_train_part,
                       eval_set=[(X_val, y_val)],
                       eval_metric='mae',
                       callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)])

        # --- Оценка модели ---
        y_train_pred_full = lgbm_model.predict(X_train)
        mae_full_train = mean_absolute_error(y_train, y_train_pred_full)
        print(f"\nMAE на полном обучающем наборе (после добавления признаков): {mae_full_train:.4f}")

        y_test_pred = lgbm_model.predict(X_test)

        # --- Создание файла submission ---
        submission_df = pd.DataFrame({'Id': test_ids, 'JP_Sales': y_test_pred})
        submission_df['JP_Sales'] = submission_df['JP_Sales'].clip(lower=0)
        submission_df.to_csv('submission.csv', index=False)

        print("Submission файл 'submission.csv' создан успешно.")

    except ValueError as e:
        print(f"\n!!! ОШИБКА при создании DataFrame или обучении модели: {e} !!!")
        print("Это часто связано с несоответствием размеров или некорректными именами признаков.")
        if 'original_feature_names' in locals():
            print("Оригинальные имена признаков (перед очисткой):", original_feature_names)
        print("Очищенные имена признаков (all_feature_names):", all_feature_names)
        print("Размер combined_processed:", combined_processed.shape)
    except LightGBMError as e:
        print(f"\n!!! LightGBM ERROR: {e} !!!")
        print("Вероятно, проблема с именами признаков, которые еще не были полностью очищены или стали дубликатами.")
        print("Проверьте вывод '--- Детальная проверка имен признаков ---' и '--- Очистка имен признаков ---'.")
        if 'original_feature_names' in locals():
            print("Оригинальные имена признаков (перед очисткой):", original_feature_names)
        print("Очищенные имена признаков (all_feature_names):", all_feature_names)
        print("Размер combined_processed:", combined_processed.shape)


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
# --- 1. Загрузка данных ---
try:
    train_df = pd.read_csv("Video_Games.csv")
    test_df = pd.read_csv("Video_Games_Test.csv")
    # sample_submission = pd.read_csv("sample_submission.csv") # Если есть файл с примером
except FileNotFoundError:
    print("Убедитесь, что файлы CSV находятся в той же директории.")
    exit()

# Сохраняем Id для тестовой выборки
test_ids = test_df['Id']

# Добавляем столбец is_train и Id к обучающей выборке для удобства объединения
# и последующего разделения. Id здесь не используется для обучения,
# но важен для финального формирования submission.
# Для обучающей выборки, 'Id' может быть NaN, но мы их не будем использовать.
train_df['is_train'] = 1
test_df['is_train'] = 0

# Объединяем для удобства предобработки
# Добавляем столбец 'JP_Sales' к тестовой выборке как NaN, чтобы конкатенация прошла
test_df['JP_Sales'] = np.nan

combined_df = pd.concat([train_df, test_df], ignore_index=True)

In [3]:
# --- 2. Исследование данных и предобработка ---
print("Информация о данных:")
combined_df.info()

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         11703 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Critic_Score     8137 non-null   float64
 10  Critic_Count     8137 non-null   float64
 11  User_Score       7590 non-null   float64
 12  User_Count       7590 non-null   float64
 13  Developer        10096 non-null  object 
 14  Rating           9950 non-null   object 
 15  is_train         16719 non-null  int64  
 16  Id               5016 non-null   floa

In [4]:
# Обработка пропущенных значений
# Name и Genre - очень мало пропусков, заменим на 'Unknown'
combined_df['Name'] = combined_df['Name'].fillna('Unknown')
combined_df['Genre'] = combined_df['Genre'].fillna('Unknown')

In [5]:
# Publisher - заменим на 'Unknown'
combined_df['Publisher'] = combined_df['Publisher'].fillna('Unknown')

In [6]:
# Year_of_Release - заменим на медиану, так как это числовой признак
median_year = combined_df['Year_of_Release'].median()
combined_df['Year_of_Release'] = combined_df['Year_of_Release'].fillna(median_year)

In [7]:
# Critic_Score, User_Score - заменим на 0 и создадим индикаторы наличия
# Эти признаки имеют много пропусков, начнем с простого подхода
combined_df['has_critic_score'] = combined_df['Critic_Score'].notna().astype(int)
combined_df['Critic_Score'] = combined_df['Critic_Score'].fillna(0)

In [8]:
combined_df['has_user_score'] = combined_df['User_Score'].notna().astype(int)
combined_df['User_Score'] = combined_df['User_Score'].fillna(0)

In [9]:
# Critic_Count, User_Count - аналогично, заменим на 0
combined_df['has_critic_count'] = combined_df['Critic_Count'].notna().astype(int)
combined_df['Critic_Count'] = combined_df['Critic_Count'].fillna(0)

In [10]:
combined_df['has_user_count'] = combined_df['User_Count'].notna().astype(int)
combined_df['User_Count'] = combined_df['User_Count'].fillna(0)

In [11]:
# Developer - заменим на 'Unknown'
combined_df['Developer'] = combined_df['Developer'].fillna('Unknown')

In [12]:
# Rating - заменим на 'Unknown'
combined_df['Rating'] = combined_df['Rating'].fillna('Unknown')

In [13]:
# Логарифмирование целевой переменной
# Важно: Применяем только к обучающей части данных, где JP_Sales известны
# Мы будем предсказывать log1p(JP_Sales), поэтому создаем новую колонку
combined_df['JP_Sales_log'] = np.log1p(combined_df['JP_Sales'])

In [14]:
# --- 3. Создание новых признаков (Feature Engineering) ---

# Признаки продаж
combined_df['Total_Sales'] = combined_df['NA_Sales'] + combined_df['EU_Sales'] + combined_df['Other_Sales']
# Добавляем небольшую константу, чтобы избежать деления на ноль
combined_df['Total_Sales_Safe'] = combined_df['Total_Sales'] + 1e-6

combined_df['NA_Sales_Ratio'] = combined_df['NA_Sales'] / combined_df['Total_Sales_Safe']
combined_df['EU_Sales_Ratio'] = combined_df['EU_Sales'] / combined_df['Total_Sales_Safe']
combined_df['Other_Sales_Ratio'] = combined_df['Other_Sales'] / combined_df['Total_Sales_Safe']

In [15]:
# Признак "возраста игры"
# Используем Year_of_Release
current_year = 2024 # Используем более актуальный год или год после всех данных
combined_df['Game_Age'] = current_year - combined_df['Year_of_Release']
combined_df['Game_Age'] = combined_df['Game_Age'].apply(lambda x: max(x, 0)) # Возраст не может быть отрицательным

In [16]:
# Десятилетие выхода игры
combined_df['Year_Decade'] = (combined_df['Year_of_Release'].astype(int) // 10) * 10

In [17]:
# --- 4. Разделение на обучающую и тестовую выборки ---
# Теперь важно разделить на основе 'is_train'
train_processed_df = combined_df[combined_df['is_train'] == 1].copy()
test_processed_df = combined_df[combined_df['is_train'] == 0].copy()

In [18]:
# Определяем признаки (X) и целевую переменную (y)
# Удаляем колонки, которые не будут использоваться как признаки
features = [col for col in train_processed_df.columns if col not in ['Id', 'Name', 'JP_Sales', 'JP_Sales_log', 'is_train']]
target = 'JP_Sales_log' # Работаем с логарифмированной целью

In [19]:
X = train_processed_df[features]
y = train_processed_df[target]
X_test = test_processed_df[features]

In [20]:
# --- 5. Предобработка категориальных признаков ---
categorical_features = ['Platform', 'Genre', 'Publisher', 'Developer', 'Rating', 'Year_Decade']

In [21]:
# Создаем трансформер
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Оставить остальные признаки без изменений
)

In [22]:
# --- 6. Выбор модели и обучение ---

# LightGBM - хороший выбор для начала
model = LGBMRegressor(random_state=42,
                      n_estimators=1500, # Увеличим количество деревьев
                      learning_rate=0.03, # Немного уменьшим learning rate
                      num_leaves=40,      # Можно поиграть с num_leaves
                      n_jobs=-1,
                      colsample_bytree=0.8, # Параметры регуляризации
                      subsample=0.8)

In [23]:
# Создаем пайплайн
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', model)])

In [24]:
# Обучение на всех данных
pipeline.fit(X, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000354 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2815
[LightGBM] [Info] Number of data points in the train set: 11703, number of used features: 191
[LightGBM] [Info] Start training from score 0.057513


,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [25]:
# --- 7. Предсказание ---
predictions_log = pipeline.predict(X_test)

/Users/marksqrt/Desktop/MTUCI/7 semestr/Системы искусственного интелекта/Video_game/venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [26]:
# Обратное преобразование (экспоненцирование)
predictions = np.expm1(predictions_log)

In [27]:
# Убедимся, что предсказания не отрицательные
predictions[predictions < 0] = 0

In [28]:
# --- 8. Формирование выходного файла ---
submission_df = pd.DataFrame({'Id': test_ids, 'JP_Sales': predictions})

# Округляем до нужного количества знаков после запятой 5
submission_df['JP_Sales'] = submission_df['JP_Sales'].round(5)

submission_df.to_csv('submission.csv', index=False)

print("\nПредсказания готовы. Файл 'submission.csv' создан.")
print(submission_df.head())


Предсказания готовы. Файл 'submission.csv' создан.
   Id  JP_Sales
0   1   0.04785
1   2   0.00000
2   3   0.00000
3   4   0.00000
4   5   0.00032
